# 2.1. Data manipulation
- 首先，我们介绍n维数组，也称为张量（tensor）。
无论使用哪个深度学习框架，它的张量类（在MXNet中为ndarray， 在PyTorch和TensorFlow中为Tensor）都与Numpy的ndarray类似。
- 但深度学习框架又比Numpy的ndarray多一些重要功能： 首先，GPU很好地支持加速计算，而NumPy仅支持CPU计算； 其次，张量类支持自动微分。

## 2.1.1 Getting Started

In [1]:
import torch
x = torch.arange(12)  # 使用 arange 创建一个行向量 x, x= tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
print(x.shape)  # torch.Size([12])
x.numel()  # 张量中元素的总数(形状的所有元素乘积), 12

torch.Size([12])


12

In [2]:
"""
通过-1来调用此自动计算出维度的功能:
    X=x.reshape(-1,4)
    X=x.reshape(3,-1)
"""

X = x.reshape(3, 4)  # X=tensor([[ 0,  1,  2,  3],
                                # [ 4,  5,  6,  7],
                                # [ 8,  9, 10, 11]])

# 创建一个形状为（2,3,4）的张量，其中所有元素都设置为0。
torch.zeros((2, 3, 4))

# 创建一个形状为（2,3,4）的张量，其中所有元素都设置为0。
torch.ones((2, 3, 4))

# 创建一个形状为（3,4）的张量。 其中的每个元素都从均值为0、标准差为1的标准高斯分布（正态分布）中随机采样。
torch.randn(3, 4)

# 通过提供包含数值的Python列表（或嵌套列表），来为所需张量中的每个元素赋予确定值。
torch.tensor([[2, 1, 4, 3],
              [1, 2, 3, 4],
              [4, 3, 2, 1]
              ])

tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [4, 3, 2, 1]])

## 2.1.2. Operations

In [3]:
# 对于任意具有相同形状的张量， 常见的标准算术运算符（+、-、*、/和**）都可以被升级为按元素运算。
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2,2,2,2,])
x+y  # (tensor([ 3.,  4.,  6., 10.])
x-y  # tensor([-1.,  0.,  2.,  6.]),
x*y  # tensor([ 2.,  4.,  8., 16.])
x/y  # tensor([0.5000, 1.0000, 2.0000, 4.0000])
x**y  # tensor([ 1.,  4., 16., 64.])
torch.exp(x)  # tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03])

# 沿行（dim=0, 轴-0，形状的第一个元素） 和按列（dim=1, 轴-1，形状的第二个元素）连结两个矩阵
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

# 通过逻辑运算符构建二元张量。
B = torch.tensor([[False,True,False,True],
                  [False,False,False,False],
                  [False,False,False,False]])
B.sum()  # tensor(2)

tensor(2)

## 2.1.3. Broadcasting

In [4]:
a = torch.arange(3).reshape((3,1))  # tensor([[0],
                                           # [1],
                                           # [2]])
b = torch.arange(2).reshape((1,2))  # tensor([[0],
                                           # [1]])
"""
矩阵a将复制列， 矩阵b将复制行，然后再按元素相加。

tensor([[0, 1],
        [1, 2],
        [2, 3]])
"""
a+b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

## 2.1.4. Indexing and Slicing
- 第一个元素的索引是0，最后一个元素索引是-1
- ":":range of row/cell
- (StartRow:numOfRow , StartCell:numOfCell)

In [5]:
X=torch.arange(12).reshape((3,4))
X[-1]  # 用[-1]选择最后一个元素, tensor([ 8,  9, 10, 11])
X[1:3]  # 用[1:3]选择第二个和第三个元素, tensor([[ 4.,  5.,  6.,  7.],
                                        #  [ 8.,  9., 10., 11.]])
X[1, 3]  # 指定索引, tensor(7)

X[0:2,:]=12
X

tensor([[12, 12, 12, 12],
        [12, 12, 12, 12],
        [ 8,  9, 10, 11]])

## 2.1.5. Saving Memory
- 运行一些操作可能会导致为新结果分配内存
    - 用Python的id()函数演示了这一点
- 首先，我们不想总是不必要地分配内存。 在机器学习中，我们可能有数百兆的参数，并且在一秒内多次更新所有参数。 通常情况下，我们希望原地执行这些更新。 其次，如果我们不原地更新，其他引用仍然会指向旧的内存位置， 这样我们的某些代码可能会无意中引用旧的参数。

In [6]:
Y=torch.arange(3).reshape((3,1))
before = id(Y)
Y=X+Y
id(Y)==before

False

In [7]:
"""
使用切片表示法将操作的结果分配给先前分配的数组，执行原地操作
- define a tensor that equal the result's shape
"""
Z=torch.zeros_like(X)
print('id(Z):',id(Z))
Z[:]=X+Y
print('id(Z):',id(Z))

# 如果在后续计算中没有重复使用X， 我们也可以使用X[:] = X + Y或X += Y来减少操作的内存开销。
before=id(X)
X+=Y  # X[:]=X+Y
id(X)==before

id(Z): 140601383282912
id(Z): 140601383282912


True

## 2.1.6 Conversion to Other Python Objects

In [8]:
A=X.numpy()
B=torch.from_numpy(A)
type(A),type(B)

(numpy.ndarray, torch.Tensor)

In [9]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)

## 2.1.7 小结
- 深度学习存储和操作数据的主要接口是张量（维数组）。它提供了各种功能，包括基本数学运算、广播、索引、切片、内存节省和转换其他Python对象。